In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
from tqdm import tqdm_notebook
import pandas as pd
from collections import defaultdict
import re
from scipy import sparse 

# поменяйте на свой путь
DS_FILE_NAME_test = r'C:\Users\DEXP\Documents\Semantix\dict.opcorpora.txt\test.txt'
DS_FILE_NAME = r'C:\Users\DEXP\Documents\Semantix\dict.opcorpora.txt\dict.opcorpora.txt'

In [36]:
def encodeSparse(word, vectorLength, symbols, symbolsLength):
    vector = np.zeros((vectorLength, 1))
    wordLength = len(word)
    for i in range(0, wordLength):
        symbol = symbols.index(word[i])
        vector[symbol + symbolsLength*i + symbolsLength*wordLength*(wordLength-1)/2] = 1
    return sparse.coo_matrix(vector)

In [16]:
lengthLongestWord = 37
lengthShortestWord = 1
lengthSymbols = 46
vectorLength = lengthSymbols*(lengthShortestWord + lengthLongestWord)/2 * (lengthLongestWord - lengthShortestWord + 1)

with open(DS_FILE_NAME, 'r') as f:            
            classes = {}
            words = {}
            symbols = set([])
            wordToClass = {}
            # прогуляемся по строкам файла
            for line in tqdm_notebook(f, mininterval=1):
                pair = line.strip().split('\t')
                if len(pair)<=1:
                    continue
                word, signs = pair[0].decode('utf-8'), re.split(' |,',pair[1])
                #print len(pair)
                symbols = symbols.union(set(word))
                pos = signs[0]
                #print word +' - ' + pos+' - ' + ','.join(sorted(set(signs[1:])))
                #print pos
                wholeclass = pos +',' + ','.join(sorted(set(signs[1:])))
                #print wholeclass
                if word not in words:
                    words[word] = len(words)
                if wholeclass not in classes:
                    classes[wholeclass] = len(classes)
                if word not in wordToClass:
                    arr = [wholeclass]
                    wordToClass[word] = arr
                else:
                    arr = wordToClass[word]
                    arr.append(wholeclass)
                    wordToClass[word] = arr
                    
            print 'classes number: ' + str(len(classes))
            print 'symbols number: ' + str(len(list(symbols)))
            print 'words number: ' + str(len(list(words)))
            print 'words with classes number: ' + str(len(list(wordToClass)))
            listSym = list(symbols)
            listSymblosLength=len(listSym)


 classes number: 4618
symbols number: 46
words number: 3039479
words with classes number: 3039479


In [75]:
print ",".join(sorted(list(symbols)))

',-,0,1,2,3,4,5,6,7,8,9,Ё,А,Б,В,Г,Д,Е,Ж,З,И,Й,К,Л,М,Н,О,П,Р,С,Т,У,Ф,Х,Ц,Ч,Ш,Щ,Ъ,Ы,Ь,Э,Ю,Я,’


In [76]:
longestWord =  max(words.keys(), key=len)
print 'Longest word: ' + longestWord
print 'Length of longest word: ' + str(len(longestWord))

Longest word: НЕЧЕРНОЗЕМАГРОСПЕЦПРОМ-МОНТАЖНАЛАДКОЙ
Length of longest word: 37


In [78]:
shortestWord =  min(words.keys(), key=len)
print 'Shortest word: ' + shortestWord
print 'Length of shortest word: ' + str(len(shortestWord))

Shortest word: З
Length of shortest word: 1


In [17]:
# Sample
listSym = list(symbols)
x = encodeSparse((u'слово').upper(), vectorLength, listSym)
print x

  (18, 0)	1.0
  (72, 0)	1.0
  (80, 0)	1.0
  (187, 0)	1.0
  (357, 0)	1.0


In [104]:
y = encodeSparse((u'слово').upper(), vectorLength, listSym)
print y

  (18, 0)	1.0
  (72, 0)	1.0
  (80, 0)	1.0
  (187, 0)	1.0
  (357, 0)	1.0


In [10]:

classes[wordToClass[u'СЛОВО'][0]]

465

In [ ]:
X=[]
yLength=len(wordToClass)
Y=[]#np.zeros((yLength, len(classes)))
for key in wordToClass:
    x = encodeSparse(key, vectorLength, listSym, listSymblosLength)
    X.append(x)
    y = np.zeros((yLength, 1))
    wordClasses = wordToClass[key]
    for gramem in wordClasses:
        y[classes[gramem]] = 1
    Y.append(sparse.coo_matrix(y))
        
    

# todo сделать кодирование слов числовыми признаками где position буквы - это набор бинарных признаков типа "индекс-длина_слова". Сами буквы кодировать бинарными признаками-тройками типа "буква-индекс-длина_слова". В таком случае бинарных признаков будет порядка: 50х(1+37)/2*38=36100. Написать формулу для кодирования произвольного слова в вектор признаков